In [15]:
# 第7章: データベース
import json

# JSONファイルからデータを読み込み
with open('artist.json', encoding='UTF-8') as data_file:
    artist_data = [json.loads(line) for line in data_file]
    
print(len(artist_data))

921337


In [27]:
# 60. KVSの構築

import redis

r = redis.StrictRedis(host='localhost', port=16379, db=0)
for artist in artist_data:
    if 'area' in artist:
        r.set(artist['name'], artist['area'])

print(len(r.keys()))

309317


In [40]:
# 61. KVSの検索

print(r.get('Oasis').decode('utf-8'))

United Kingdom


In [60]:
# 62. KVS内の反復処理
from itertools import islice

artists_in_japan = [key for key in r.keys() if r.get(key) == b'Japan']

for artist in islice(artists_in_japan, 0, 10):
    print(artist.decode('utf-8'))

print('Count : {}'.format(len(artists_in_japan)))

Mitomi Tokoto
川嶋未来
青空千夜
Kicco
さわやか五郎
杏子
中西宏司
SIMONSAYZ
天江衣
アイドルプロジェクト
Count : 22128


In [42]:
# 63. オブジェクトを値に格納したKVS

r1 = redis.StrictRedis(host='localhost', port=16379, db=1)
for artist in artist_data:
    if 'tags' in artist:
        r1.set(artist['name'], artist['tags'])

print(r1.get('GANGgajang').decode('utf-8'))

[{'count': 1, 'value': 'australian'}, {'count': 1, 'value': 'rock'}]


In [51]:
# 64. MongoDBの構築
from pymongo import MongoClient
import pymongo

# DBへ接続
client = MongoClient('localhost', 27017)
db = client.test_db
artists = db.artists # artitsのコレクションを作成・参照

# DBにデータを格納
result = artists.insert_many(artist_data)

print(len(result.inserted_ids))

index_name = artists.create_index([('name', pymongo.ASCENDING)])
print(index_name)

index_alias = artists.create_index([('aliases.name', pymongo.ASCENDING)])
print(index_alias)

index_tag = artists.create_index([('tags.value', pymongo.ASCENDING)])
print(index_tag)

index_rating = artists.create_index([('rating.value', pymongo.ASCENDING)])
print(index_rating)

921337
name_1
aliases.name_1
tags.value_1
rating.value_1


In [58]:
# 65. MongoDBの検索

artist_queen = artists.find_one({'name' : 'Queen'})

print(artist_queen)

{'gender': 'Female', '_id': ObjectId('57ad7cad3d48711c401e10d9'), 'aliases': [{'sort_name': 'Queen', 'name': 'Queen'}], 'ended': True, 'sort_name': 'Queen', 'area': 'Japan', 'type': 'Character', 'name': 'Queen', 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', 'id': 701492, 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}]}


In [61]:
# 66. 検索件数の取得

artists_in_japan_2 = artists.find({'area': 'Japan'})

print(artists_in_japan_2.count())

22821


In [67]:
# 67. 複数のドキュメントの取得

artists_oasis = artists.find({'aliases.name' : 'オアシス'})

for artist in artists_oasis:
    print(artist)

{'gid': '39ab1aed-75e0-4140-bd47-540276886b60', 'id': 20660, 'begin': {'year': 1991}, 'sort_name': 'Oasis', 'name': 'Oasis', 'type': 'Group', 'area': 'United Kingdom', 'ended': True, '_id': ObjectId('57ad7cae3d48711c401efc98'), 'aliases': [{'sort_name': 'OASIS', 'name': 'OASIS'}, {'sort_name': 'オアシス', 'name': 'オアシス'}], 'rating': {'count': 13, 'value': 86}, 'end': {'date': 28, 'month': 8, 'year': 2009}, 'tags': [{'count': 1, 'value': 'rock'}, {'count': 3, 'value': 'britpop'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'uk'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'rock and indie'}, {'count': 1, 'value': 'england'}, {'count': 1, 'value': 'manchester'}]}


In [84]:
# 68. 複数のドキュメントの取得

artists_dance = artists.aggregate([
        { '$match' : {'tags.value' : 'dance', 'rating.value' : {'$exists' : 'true' }}},
        { '$sort' : {'rating.value' : -1 }}, #descending
        { '$limit': 10 }
    ])

for artist in artists_dance:
    print('name = {}, rating = {}'.format(artist['name'], artist['rating']['value']))

name = Susie Ledge, rating = 100
name = Miami Horror, rating = 100
name = Annelise Collette, rating = 100
name = Neneh Cherry, rating = 100
name = slowEarth, rating = 100
name = David Guetta, rating = 100
name = Maxi Jazz, rating = 100
name = Sonny Fodera, rating = 100
name = RJ. Ruben Junnior, rating = 100
name = Mountane, rating = 100


In [ ]:
# 69. Webアプリケーションの作成

